In [ ]:
import torch
import torch.nn as nn


def noise_images(x, t):
    beta = torch.linspace(1e-4, 0.02, 1000)
    alpha = 1. - beta
    alpha_hat = torch.cumprod(alpha, dim=0)

    sqrt_alpha_hat = torch.sqrt(alpha_hat[t])[:, None, None, None]
    sqrt_one_minus_alpha_hat = torch.sqrt(1. - alpha_hat[t])[:, None, None, None]

    epsilon = torch.rand_like(x)
    return sqrt_alpha_hat * x + sqrt_one_minus_alpha_hat * epsilon, epsilon


def pos_encoding(t, channels):
    even_inv_freq = 1.0 / (10000**(torch.arange(0, channels, 2).float() / channels))
    odd_inv_freq = 1.0 / (10000**(torch.arange(1, channels, 2).float() / channels))

    pos_even_enc_a = torch.sin(t.repeat(1, channels // 2) * even_inv_freq)
    pos_odd_enc_b = torch.cos(t.repeat(1, channels // 2) * odd_inv_freq)

    pos_enc = torch.cat([pos_even_enc_a, pos_odd_enc_b], dim=-1)
    return pos_enc


In [ ]:
t = torch.randint(low=1, high=1000, size=(5,))
images = torch.randn(5, 64, 64)
x_t, noise = noise_images(images, t)
# x_t shape is (5,5,64,64)
# noise shape is (5,64,64)

In [ ]:
time_dim, emb_dim = 256, 256
out_channels = 128
t = t.unsqueeze(-1).type(torch.float)
t = pos_encoding(t, time_dim) # (5,256)

In [ ]:
emb_layer = nn.Sequential(nn.SiLU(),nn.Linear(emb_dim, out_channels))
emb_layer(t)

In [ ]:
x_t.shape

In [ ]:
channels = 5
size = 16

x_t.view(-1, channels, size * size).swapaxes(1,2).shape
# 

In [ ]:
layer = nn.LayerNorm([5])
x = torch.randn(5, 5, 64, 64)
k = torch.randn(80,256,5)
layer(k).shape
self_mha = nn.MultiheadAttention(5, 5, batch_first=True)
self_mha(k,k,k)[0] + k

ff_self = nn.Sequential(
    nn.LayerNorm([5]),
    nn.Linear(5, 5),
    nn.GELU(),
    nn.Linear(5, 5)
)

In [ ]:
# ff_self(self_mha(k,k,k)[0] + k).shape
# 
torch.randn(80,256,5).swapaxes(2, 1).view(-1, 5, 16, 16).shape

In [ ]:
all_images_in_this_epoch = torch.tensor(())

In [ ]:
image1 = images[0]
image2 = images[1]
image2.shape

In [ ]:
all_images_in_this_epoch = torch.cat((all_images_in_this_epoch, images), 0)
all_images_in_this_epoch

In [ ]:
all_images_in_this_epoch = torch.cat((all_images_in_this_epoch, image2), -2)
all_images_in_this_epoch

In [ ]:
from datasets import load_dataset
ds = load_dataset("uoft-cs/cifar10")
ds

In [ ]:
next(iter(DataLoader(ds['train'], batch_size=12, shuffle=True)))

In [ ]:
import torchvision.transforms as transforms

imggg = ds['train']['img'][0]
transform = transforms.Compose([transforms.ToTensor()])
tensor = transform(imggg)
tensor.shape

In [ ]:
from torch.utils.data import DataLoader

loader = DataLoader(ds['train'], batch_size=12, shuffle=True)
for train_features in loader:
    print(train_features)
    # print(train_labels)

In [ ]:
import torch
# import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms

transform = transforms.Compose([transforms.ToTensor()])

tensor_x = torch.stack(list(map(lambda PILimg: transform(PILimg), ds['train']['img'])))
tensor_y = torch.Tensor(ds['train']['label'])

my_dataset = TensorDataset(tensor_x, tensor_y) # create your datset
my_dataloader = DataLoader(my_dataset)

In [ ]:
for train_features, train_labels in my_dataloader:
    print(train_features)
    print(train_labels)
    break

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
import torch.nn as nn
from torch.nn import functional as F
import torch

class DoubleConv(nn.Module):
    '''
    Used in initial convolution of images as well as 
    at the bottom of the the U shape net.
    '''
    def __init__(self, in_channels, out_channels, mid_channels=None, residual=False):

        super().__init__()
        self.residual = residual
        self.normalizing_group_num = 1
        if not mid_channels:
            mid_channels = out_channels
        
        ## The following layers are applied sequentially in the
        ## defined order as follows.
        self.double_conv = nn.Sequential(

            ## Weights of conv are learned through backpropagation
            ## We are transforming the channels number from in_channels to mid_channels
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),

            ## We normalize the values group wise by separating the channels
            ## the group number. In this case it is 1. Output's shape
            ## remains unchanged from the input shape.
            nn.GroupNorm(self.normalizing_group_num, mid_channels),

            ## Apply Gaussian error linear units
            nn.GELU(),

            ## We are transforming the channels number from mid_channels to out_channels
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),

            ## Perform group normalization once again
            ## n×c×w×h -> swh×gn where c = sg
            nn.GroupNorm(self.normalizing_group_num, out_channels)
        )


    def forward(self, x):
        ## We apply gelu when we have skip connections to the UNet decoder
        if self.residual:
            return F.gelu(x + self.double_conv(x))
        return self.double_conv(x) 
    

class newer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.inc = DoubleConv(in_channels, out_channels)
    

a = newer(10,20)
# a.inc[0]

nn.ModuleList([newer(10,687), DoubleConv(456, 7)])[0](torch.randn(5, 10, 687))



In [ ]:
a.load_state_dict(torch.load(PATH), strict=True)
a.double_conv

In [ ]:
import os
os.path.exists('./model.pt')

In [ ]:
for p in model.parameters():
    p.detach().zero_()
    print(p)

In [ ]:
import copy
class EMA:
    def __init__(self, beta):
        self.beta = beta
        self.step = 0

    def update_model_average(self, ema_model, model):
        for curr_param, ema_param in zip(model.parameters(), ema_model.parameters()):
            old_weight, new_weight = ema_param.data, curr_param.data
            ema_param.data = self.update_average(old_weight, new_weight)
            
    ## We have a apply the smoothing where the EMA model
    ## parameters will be updated by the moving average
    ## of the old and new weights.
    def update_average(self, old_weight, new_weight):
        return old_weight * self.beta + (1 - self.beta) * new_weight
    

    def reset_parameters(self, ema_model, model):
        ema_model.load_state_dict(model.state_dict())


    ## At the start, even as we train the non ema model, 
    ## we just allow the ema model take copy the learnt weights
    ## from the non ema model. We don't want to over average early
    ## as this overemphasizes the the early stage weights.
    ## For simplicity, we start the averaging after 2000 steps
    ## of non ema model training (assumes that our data and 
    ## no. of epochs will exceed and best be more 2x more than 2000)
    def step_ema(self, ema_model, model, step_start_ema=2000):
        if self.step < step_start_ema:
            self.reset_parameters(ema_model, model)
            self.step += 1
            return
        self.update_model_average(ema_model, model)
        self.step += 1

ema = EMA(beta=0.995)
ema_model = copy.deepcopy(model).eval().requires_grad_(False)

In [ ]:
copy.deepcopy(model).eval()

In [ ]:
class Parent1:
    def __init__(self, a, b, c=2):
        self.a = a
        self.b = b

        if c < 3:
            self.c = 567567
    
    def aaa(self, neww):
        return neww
    

class Parent2:
    def __init__(self, k=1, j=56):
        self.k = k
        self.j = j

# class childB(Parent2, Parent1):
#     def __init__(self, i):
#         super().__init__(b=3,a=4)
#         self.i = i
            
#     def bbb(self, neww):
#         return neww + 345


class childA(Parent2, Parent1):
    def __init__(self, x):
        super().__init__()
        Parent1.__init__(self, a=3, b=2)
        # Parent2.__init__(self, 657, 789)
        self.x = x
    
    def help(self):
        print(self.k)
        return self.aaa(self.c)

a = childA(546)
a.help()

In [ ]:
import cv2
import torchvision

# # Load the image
# image = cv2.imread("img1.jpeg")

img = cv2.imread('messi5.jpeg', cv2.IMREAD_GRAYSCALE)
img = cv2.GaussianBlur(img, (5,5), 0)

transforms.Resize


img_tensor = torch.tensor(img)
print(img_tensor.unsqueeze(0).shape)
image = torchvision.transforms.ToPILImage()(img_tensor.unsqueeze(0))

out = cv2.Canny(img, 50, 200, L2gradient=True)
out

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms
from datasets import load_dataset

def get_data_loader(batch_size, is_data_loader_shuffle):

    ## Load the cifar dataset for a set of images 
    ## from huggingface for testing purposes
    ds = load_dataset("uoft-cs/cifar10")

    ## Extra operations to convert data into a dataset and
    ## fed in dataloader
    transform = transforms.Compose([transforms.ToTensor()])
    tensor_x = torch.stack(list(map(lambda PILimg: transform(PILimg), ds['train']['img'])))
    tensor_y = torch.LongTensor(ds['train']['label'])

    my_dataset = TensorDataset(tensor_x, tensor_y)
    my_dataloader = DataLoader(my_dataset,
                               batch_size=batch_size,
                               shuffle=is_data_loader_shuffle)
    return my_dataloader

In [ ]:
for images, labels in get_data_loader(12, True):
    print(images.shape)
    print(images.unsqueeze(0).shape)
    tt = transforms.Compose([transforms.ToPILImage()])
    image_jpg = tt(images)
    break


img.shape

In [ ]:
import torch
import cv2
import torchvision.transforms as transforms
import torchvision.transforms.functional as F
import numpy as np
import matplotlib.pyplot as plt

def grey_and_blur(image):
    # images_gray = np.mean(image.numpy(), axis=1)

    # Normalize the grayscale image to the range [0, 255]
    images_gray_normalized = cv2.normalize(image.numpy(), None, 0, 255, cv2.NORM_MINMAX)
    # grayscale_trans = transforms.Grayscale(num_output_channels=1)
    blurred_image = cv2.GaussianBlur(images_gray_normalized, (3,3), 0)
    blurred_image = blurred_image.astype(np.uint8)
    return cv2.Canny(blurred_image, 50, 150)


for images, labels in get_data_loader(12, True):
    
    cv2.imshow('Original', images[0].numpy())
     
    tt = transforms.Compose([
        transforms.Lambda(lambda x: [grey_and_blur(img) for img in x])
    ])
    image_jpg = tt(images)
    # print(image_jpg[0].shape)
    # plt.subplot(122), cv2.imshow(torch.tensor(image_jpg[0]))
    break


# img = cv2.imread('messi5.jpeg', cv2.IMREAD_GRAYSCALE)
# img = cv2.GaussianBlur(img, (5,5), 0)

# img_tensor = torch.tensor(img)
# print(img_tensor.unsqueeze(0).shape)
# image = torchvision.transforms.ToPILImage()(img_tensor.unsqueeze(0))
# out = cv2.Canny(img, 50, 200, L2gradient=True)
# out





# np.nonzero(out)

In [ ]:
import cv2 as cv
from matplotlib import pyplot as plt
import numpy

for imgggg in load_dataset("uoft-cs/cifar10")['train']['img']:

    new_img = numpy.asarray(imgggg)
    (B, G, R) = cv2.split(new_img)

    gray = cv.cvtColor(new_img, cv.COLOR_BGR2GRAY)

    # edges_new = cv.Canny(gray,100,200)

    B_cny = cv2.Canny(B, 100, 200)
    G_cny = cv2.Canny(G, 100, 200)
    R_cny = cv2.Canny(R, 100, 200)

    edges_new = cv2.merge([B_cny, G_cny, R_cny])
    # edges_new = np.transpose(edges_new, (2, 0, 1))
    print(torch.tensor(edges_new).shape)

    im_rgb = cv.cvtColor(new_img, cv.COLOR_BGR2RGB)
    print(im_rgb.shape)
    plt.subplot(121),plt.imshow(im_rgb,cmap = 'Blues')
    plt.title('Original Image'), plt.xticks([]), plt.yticks([])

    plt.subplot(122),plt.imshow(edges_new, cmap = 'bone')
    plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
    break

In [ ]:
ds = load_dataset("uoft-cs/cifar10")

def get_canny_edges(PILimg):
    trans_img = numpy.asarray(PILimg)

    (B, G, R) = cv2.split(trans_img)
    B_cny = cv2.Canny(B, 100, 200)
    G_cny = cv2.Canny(G, 100, 200)
    R_cny = cv2.Canny(R, 100, 200)
    edges = cv2.merge([B_cny, G_cny, R_cny])
    edges = np.transpose(edges, (2, 0, 1))
    return edges

transform = transforms.Compose([transforms.ToTensor()])
tensor_x = torch.stack(list(map(lambda PILimg: transform(PILimg), ds['train']['img'])))
tensor_y = torch.LongTensor(ds['train']['label'])
tensor_z = torch.stack(list(map(lambda PILimg: torch.tensor(get_canny_edges(PILimg)), ds['train']['img'])))

my_dataset = TensorDataset(tensor_x, tensor_y, tensor_z) # create your datset
my_dataloader = DataLoader(my_dataset,
                           batch_size=12,
                           shuffle=True)


for i, (images, labels, edges) in enumerate(my_dataloader):
    print(images.shape)
    print(edges.shape)
    print(labels.shape)

    break




In [ ]:
torch.randn(12,32,32).shape

In [ ]:
import torch.nn as nn
new_conv = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=(1,1)),
            nn.SiLU(),
            nn.Conv2d(64, 128, kernel_size=3, padding=(1,1)),
            nn.SiLU(),
            nn.Conv2d(128, 64, kernel_size=3, padding=(1,1)),
            nn.SiLU(),
            nn.Conv2d(64, 64, kernel_size=1, padding=0)
        )

new_conv(torch.randn(12, 3, 32, 32))

In [ ]:
import cv2 as cv
from matplotlib import pyplot as plt

img = cv.imread('messi5.jpeg')
print(type(img))
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
edges = cv.Canny(gray,100,200)

im_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
plt.subplot(121),plt.imshow(im_rgb,cmap = 'Blues')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(edges,cmap = 'bone')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

plt.show()

In [ ]:

# import torchvision

# # torch.tensor(img)
# aa = torchvision.transforms.Grayscale(torch.tensor(img))
# aa.numpy()

torch.randn(5,32,32).numpy()

In [3]:
import torch.nn as nn
import torch


# torch.Size([12, 256])
# torch.float32
# torch.Size([12])
# torch.int64


# torch.Size([36, 256])
# torch.float32
# torch.Size([36])
# torch.float32


aaaa = nn.Embedding(10, 256)
# aa.weight.requires_grad = False
bbb = torch.randn(12)


# b.type()

# b.weight.requires_grad = False
print(aaaa.shape)
aaaa(bbb.to(torch.int64))
# # b.type()
# # b
# print(aa.weight.requires_grad)

AttributeError: 'Embedding' object has no attribute 'shape'